# C1 - Introduction to Human and Computer Vision
## Week 4

In [56]:
import os
import re
import matplotlib.pyplot as plt
import cv2
import pickle
import numpy as np

# Get project's root directory
BASE_PATH = os.path.join(re.search(r'.+(Team5)', os.getcwd())[0], 'week4')
os.chdir(BASE_PATH)
BASE_PATH

DATA_DIRECTORY = '../data'

In [44]:
# Read pickle file to see detailed info of the images augmentation
with open(f'{DATA_DIRECTORY}/qsd1_w4/augmentations.pkl', 'rb') as f:
    augmentations_info = pickle.load(f)

# Read pickle file with correspondences
with open(f'{DATA_DIRECTORY}/qsd1_w4/gt_corresps.pkl', 'rb') as f:
    ground_truth = pickle.load(f)

### a) Remove background, detect noise (and filter it)

In [57]:
from src.background_removal import background_removal
from src.noise_removal import denoise_image
from tqdm import tqdm

# Image names
QSD1_w4_names = [f for f in os.listdir(f'{DATA_DIRECTORY}/qsd1_w4/') if f.endswith('.jpg')]
QSD1_w4_names.sort()

# Initialize datasets
BBDD = []
QSD1_w4 = []
QSD1_w4_filtered = []
QSD1_w4_nonAugmented = []

# Load datasets (+ filter)
for image_name in tqdm(QSD1_w4_names):
    # Read QSD1_w4
    image_qsd1 = cv2.imread(f'{DATA_DIRECTORY}/qsd1_w4/{image_name}')
    QSD1_w4.append(image_qsd1)

    # Read non-augmented image
    image_nonAugmented = cv2.imread(f'{DATA_DIRECTORY}/qsd1_w4/non_augmented/{image_name}')
    image_nonAug_bckg_remov = background_removal(denoise_image(image_nonAugmented))  # Remove background in non-augmented image
   
    QSD1_w4_nonAugmented.append(image_nonAug_bckg_remov)

    # Filter image from QSD1_w4
    filtered_image = background_removal(denoise_image(image_qsd1))  # Detect noise (and clean it) + Remove background
    QSD1_w4_filtered.append(filtered_image)


# Read BBDD
BBDD_names = [f for f in os.listdir(f'{DATA_DIRECTORY}/BBDD/') if f.endswith('.jpg')]
BBDD_names.sort()

for image_name in tqdm(BBDD_names):
    image_bbdd = cv2.imread(f'{DATA_DIRECTORY}/BBDD/{image_name}')
    BBDD.append(image_bbdd)

100%|██████████| 287/287 [00:03<00:00, 83.03it/s] 


In [ ]:
# Plot some examples
img_number = 17


fig, axes = plt.subplots(1, 3)  # 1 fila, 3 columnas

# QSD1_w4 image
image = QSD1_w4[img_number]
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
axes[0].imshow(image)
axes[0].set_title('QSD1_w4 image')
axes[0].axis('Off')

# Filtered image (background removal + denoise)
image = QSD1_w4_filtered[img_number][0]
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
axes[1].imshow(image)
axes[1].set_title('Filtered image')
axes[1].axis('Off')

# Non-augmented image
image = QSD1_w4_nonAugmented[img_number][0]
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
axes[2].imshow(image)
axes[2].set_title('Non-augmented image')
axes[2].axis('Off')

plt.show()
plt.close()

### b) Detect keypoints and compute descriptors

In [ ]:
'''
flags = {
  DEFAULT = 0,
  DRAW_OVER_OUTIMG = 1,
  NOT_DRAW_SINGLE_POINTS = 2,
  DRAW_RICH_KEYPOINTS = 4
}
'''
def draw_keypoints(image, kp, flags=0):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img2 = cv2.drawKeypoints(gray_image, kp, None,(255,0,0),flags=flags)
    plt.imshow(img2) 

#### SIFT

In [ ]:
def sift_descriptor(image, params={}):
    '''
    Compute SIFT descriptors for a given image
    :param image: image to compute the descriptors
    :param params: parameters for the SIFT algorithm
    :return: keypoints and descriptors

    default params = {
        'nfeatures': 0,
        'nOctaveLayers': 3,
        'contrastThreshold': 0.04,
        'edgeThreshold': 10,
        'sigma': 1.6
    }
    '''
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create(**params)
    kp, des = sift.detectAndCompute(img_gray, None)

    return (kp, des)

In [ ]:

params = {
    'nfeatures': 1500,
    'nOctaveLayers': 3,
}

sift_query = []
for picture in tqdm(QSD1_w4_filtered):
    res = []
    for painting in picture:
        kp, des = sift_descriptor(painting, params=params)
        res.append({'kp': kp, 'des': des})
    sift_query.append(res)


sift_bd = []
for painting in tqdm(BBDD):
    kp, des = sift_descriptor(painting, params=params)
    sift_bd.append({'kp': kp, 'des': des})

In [ ]:
idx1 = 16 # Query image
idx2 = 11 # BBDD image

des1 = sift_query[idx1][0]['des']
des2 = sift_bd[idx2]['des']

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
# Match descriptors.
matches = bf.match(des1,des2)
# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)
# Draw first 10 matches.
img3 = cv2.drawMatches(
    QSD1_w4_filtered[idx1][0], sift_query[idx1][0]['kp'],
    BBDD[idx2], sift_bd[idx2]['kp'],
    matches[:20], None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)
print('Distance:', np.mean([m.distance for m in matches]))
plt.imshow(img3), plt.axis('off'), plt.show()

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

res = []
dist_res = []
for i in tqdm(range(30)):
    picture_res = []
    picture_distances = []
    for j in range(0, len(sift_query[i])):
        distances = []
        for k in range(0, len(sift_bd)):
            if sift_bd[k]['des'] is not None:
                matches = bf.match(sift_query[i][j]['des'], sift_bd[k]['des'])
                matches = sorted(matches, key = lambda x:x.distance)
                # Take top 20 matches
                distance = np.mean([match.distance for match in matches[:20]])
                distances.append(distance)
            else:
                distances.append(10000)

        # Take top 10 matches
        most_similar = np.argsort(distances)[:10]
        most_similar_distances = [distances[idx] for idx in most_similar]
        if most_similar_distances[0] == 10000:
            most_similar = [-1]
            most_similar_distances = [10000]
        if (
            most_similar_distances[1] - most_similar_distances[0] < 
            2*np.mean(np.array(most_similar_distances[2:10]) - np.array(most_similar_distances[1:9]))
        ):
            most_similar = [-1]
        picture_res.append(most_similar)
        picture_distances.append(most_similar_distances)
    
    res.append(picture_res)
    dist_res.append(picture_distances)


In [ ]:
image = QSD1_w4_filtered[24][0]
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
print(res)

In [ ]:
import importlib
import src.metrics  # re-import the module to make sure it's recognized
importlib.reload(src.metrics)

from src.metrics import mapk

# Now you can call the updated mapk function
mapk(ground_truth, res, k=1)


#### HOG

In [70]:
from skimage.feature import hog
from skimage import exposure
from scipy.spatial.distance import euclidean
import importlib

import src.metrics 
importlib.reload(src.metrics)
from src.metrics import mapk

params={
    'shape': (100,100),  # Shape we want to resize the image to
    'pixels_per_cell': (18,18),
    'cells_per_block': (3,3),
}

def hog_descriptor(image, shape: tuple, pixels_per_cell: tuple, cells_per_block: tuple):
    
    # Image to grayscale
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Resize image (we need same dimensions in all the image descriptros to compare)
    image = cv2.resize(image, shape, interpolation=cv2.INTER_AREA)

    # Compute HOG
    hog_descriptor, hog_image = hog(image, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True)
    
    # Rescale intensity, otherwise we may see everyhing black
    hog_image = exposure.rescale_intensity(hog_image, in_range=(0, np.max(hog_image)/10))

    return hog_descriptor, hog_image

# Missing taking into account that there's an unknown class
def compute_results_hog(hog_query, hog_bd, k=5):
    res = []
    distances_res = []

    for i in tqdm(range(30)):
        picture_res = []
        picture_distances = []
        
        # For each painting inside an image from hog_query[i]
        for j in range(len(hog_query[i])):
            distances = []

            query_descriptor = hog_query[i][j]['descriptor']
            for p in range(len(hog_bd)): 
                bd_descriptor = hog_bd[p]['descriptor']
                distance = euclidean(query_descriptor, bd_descriptor)
                distances.append((distance, p))

            distances.sort(key=lambda x: x[0])

            nearest_neighbors = [idx for _, idx in distances[:k]] 
            nearest_distances = [round(dist, 2) for dist, _ in distances[:k]]

            picture_res.append(nearest_neighbors)
            picture_distances.append(nearest_distances)

        res.append(picture_res)
        distances_res.append(picture_distances)

    return res, distances_res


In [71]:
# Take HOG descriptor from Query and BBDD images

hog_query = []
for picture in tqdm(QSD1_w4_filtered):
    res = []
    for painting in picture:    
        des, image = hog_descriptor(painting, **params)
        res.append({'descriptor': des, 'image': image})
    hog_query.append(res)

hog_bd = []
for painting in tqdm(BBDD):    
    des, image = hog_descriptor(painting, **params)
    hog_bd.append({'descriptor': des, 'image': image})


100%|██████████| 287/287 [00:04<00:00, 64.04it/s]


In [81]:
# Compute results
res, distances_res = compute_results_hog(hog_query, hog_bd)

unknown_images_indices = [index for index, value in enumerate(ground_truth) if value == [-1]]

distances_unknown = [distances_res[index] for index in unknown_images_indices]
# Calculamos la desviación estándar de cada sublista
deviation_stds = []
for sublist in distances_unknown:
    if sublist:  # Verificar que la sublista no esté vacía
        std = np.std(sublist[0])  # Tomamos el primer (y único) elemento de la sublista
        deviation_stds.append(std)
    else:
        deviation_stds.append(None)  # Si la sublista está vacía, asignamos None


# Obtener los índices de ground_truth que no son [-1]
known_images_indices = [index for index, value in enumerate(ground_truth) if value != [-1]]

# Obtener las sublistas correspondientes de distances_res
distances_known = [distances_res[index] for index in known_images_indices]

# Calculamos la desviación estándar de cada sublista para las imágenes conocidas
deviation_stds_known = []
for sublist in distances_known:
    if sublist:  # Verificar que la sublista no esté vacía
        std = np.std(sublist[0])  # Tomamos el primer (y único) elemento de la sublista
        deviation_stds_known.append(std)
    else:
        deviation_stds_known.append(None)  # Si la sublista está vacía, asignamos None

# Mostrar los resultados
print(deviation_stds_known)

# Mostrar los resultados
print(deviation_stds)

print(ground_truth)
print(res)
print(distances_unknown)

#mapk1 = mapk(ground_truth, res, k=1)

100%|██████████| 30/30 [00:00<00:00, 362.33it/s]

[None, 0.2715437349673161, 0.33558307466259374, 0.2813112155602759, 0.3311193138432127, 0.1644384383287557, 0.24311314238436385, 0.24011663832396127, 0.32418513229326235, 0.2616409753842085, 0.23122283624244383, 0.30806492822130854, 0.14496896219536093, 0.454779067240347, 0.4673157390886808, 0.5063753548505299, 0.33941125496954283, 0.17417232845661793, 0.22199099080818574, 0.22821042920953455]
[0.05564171097297419, 0.04621688003316541, 0.11128342194594847, 0.0733212111192935, 0.034292856398964525, None, 0.07440430095095306, 0.04317406628984579, 0.14260434775980713, 0.06852736679604723]
[[-1], [150], [48, 251], [32], [161], [81], [62, 38], [-1], [128], [155, 258], [136, 76], [-1], [-1], [53], [-1], [12], [11, 280], [-1], [182, 252], [-1], [272, 117], [-1], [242], [260], [94, 132], [223], [-1], [127], [47, 13], [-1]]
[[[186, 264, 283, 11, 45]], [], [[48, 57, 60, 205, 104], [251, 51, 57, 215, 281]], [[32, 88, 82, 161, 106]], [[161, 40, 137, 205, 163]], [[81, 205, 16, 119, 25]], [[62, 64, 

In [69]:
import itertools
import cv2
from tqdm import tqdm

# Definir combinaciones de parámetros
shapes = [(100,100), (90,90)]
pixels_per_cells = [(18, 18), (16, 16), (15,15), (17,17)]
cells_per_blocks = [(2, 2), (3,3), (8,8)]

# Inicializar lista de resultados
resultados = []

# Bucle para probar todas las combinaciones de parámetros
for shape, pixels_per_cell, cells_per_block in itertools.product(shapes, pixels_per_cells, cells_per_blocks):
    try:
        # Configuración de parámetros actuales
        params = {
            'shape': shape,
            'pixels_per_cell': pixels_per_cell,
            'cells_per_block': cells_per_block,
        }

        # Aplicar HOG en imágenes de consulta (Query)
        hog_query = []
        for picture in tqdm(QSD1_w4_filtered):
            res = []
            for painting in picture:
                # Redimensionar la imagen según el parámetro `shape`
                painting_resized = cv2.resize(painting, params['shape'])
                # Calcular el HOG con los parámetros actuales
                des, image = hog_descriptor(painting_resized, **params)
                res.append({'descriptor': des, 'image': image})
            hog_query.append(res)

        # Aplicar HOG en imágenes de la base de datos (BBDD)
        hog_bd = []
        for painting in tqdm(BBDD):
            painting_resized = cv2.resize(painting, params['shape'])
            des, image = hog_descriptor(painting_resized, **params)
            hog_bd.append({'descriptor': des, 'image': image})

        # Calcular los resultados de similitud usando HOG
        res, distances_res = compute_results_hog(hog_query, hog_bd)
        score = mapk(ground_truth, res, k=1)  # Calcula la puntuación para esta combinación

        # Guardar la combinación y su puntaje
        resultados.append((params, score))

    except ValueError as e:
        print(f"Combinación inválida {params} debido a error: {e}")
        continue  # Pasar a la siguiente combinación en caso de error

# Ordenar los resultados por puntaje (de mayor a menor)
resultados.sort(key=lambda x: x[1], reverse=True)

# Guardar los mejores resultados en un archivo
with open("mejores_resultados.txt", "w") as f:
    for params, score in resultados:
        f.write(f"Parámetros: {params}, Score: {score}\n")

print("Búsqueda de parámetros completada. Resultados guardados en 'mejores_resultados.txt'.")

100%|██████████| 30/30 [00:00<00:00, 477.38it/s]


-1 [11, 264, 186, 24, 170]
48 [48, 244, 60, 140, 57]
251 [251, 51, 96, 11, 215]
32 [32, 11, 82, 193, 109]
161 [161, 40, 241, 174, 244]
81 [81, 60, 16, 40, 119]
62 [62, 64, 30, 129, 106]
38 [38, 67, 102, 52, 90]
-1 [264, 186, 11, 101, 24]
128 [128, 46, 241, 238, 228]
155 [155, 92, 40, 205, 65]
258 [258, 286, 223, 154, 163]
136 [136, 271, 37, 220, 161]
76 [76, 142, 65, 43, 16]
-1 [93, 78, 16, 156, 21]
-1 [206, 45, 283, 264, 186]
53 [53, 93, 54, 25, 276]
-1 [197, 156, 146, 200, 119]
12 [12, 221, 21, 78, 187]
11 [11, 184, 282, 255, 270]
280 [280, 4, 235, 215, 31]
182 [182, 133, 252, 259, 57]
252 [225, 252, 51, 223, 215]
-1 [16, 197, 241, 156, 205]
272 [272, 277, 116, 252, 270]
117 [117, 104, 60, 51, 42]
-1 [40, 46, 60, 23, 137]
242 [242, 74, 176, 93, 157]
260 [260, 163, 29, 133, 271]
94 [94, 225, 234, 215, 51]
132 [132, 57, 218, 223, 137]
223 [223, 58, 215, 57, 29]
-1 [93, 78, 221, 74, 12]
127 [127, 11, 241, 156, 40]
47 [47, 100, 102, 192, 279]
13 [13, 52, 100, 47, 102]
-1 [206, 135, 163, 

100%|██████████| 30/30 [00:00<00:00, 446.37it/s]


-1 [186, 264, 24, 11, 29]
48 [48, 244, 57, 140, 286]
251 [251, 51, 96, 215, 259]
32 [32, 11, 88, 82, 275]
161 [161, 40, 11, 241, 174]
81 [81, 140, 60, 241, 105]
62 [62, 64, 30, 129, 79]
38 [38, 67, 102, 52, 264]
-1 [264, 186, 283, 24, 11]
128 [128, 46, 176, 99, 241]
155 [155, 119, 31, 113, 60]
258 [258, 223, 91, 286, 163]
136 [136, 37, 271, 220, 133]
76 [76, 249, 142, 43, 65]
-1 [93, 205, 78, 221, 16]
-1 [283, 45, 206, 144, 29]
53 [53, 54, 254, 55, 198]
-1 [156, 197, 146, 200, 119]
12 [12, 221, 21, 78, 187]
11 [11, 255, 36, 170, 282]
280 [280, 4, 141, 154, 204]
182 [182, 133, 281, 259, 144]
252 [252, 215, 225, 118, 192]
-1 [106, 205, 241, 197, 16]
272 [272, 277, 154, 11, 281]
117 [117, 233, 60, 11, 40]
-1 [60, 104, 57, 31, 205]
242 [242, 93, 176, 157, 74]
260 [260, 29, 271, 163, 162]
94 [94, 225, 261, 234, 192]
132 [132, 57, 218, 137, 163]
223 [223, 215, 57, 174, 91]
-1 [93, 221, 74, 12, 21]
127 [127, 174, 11, 40, 241]
47 [47, 100, 102, 192, 225]
13 [13, 100, 102, 261, 52]
-1 [283, 45,

  0%|          | 0/30 [00:00<?, ?it/s]


...

-1 [264, 259, 135, 186, 11]
48 [48, 119, 244, 106, 88]
251 [251, 96, 11, 51, 40]
32 [32, 11, 109, 161, 82]
161 [161, 40, 16, 274, 81]
81 [81, 241, 197, 161, 16]
62 [30, 62, 129, 133, 147]
38 [38, 52, 101, 264, 91]
-1 [264, 259, 29, 174, 101]
128 [128, 238, 107, 187, 46]
155 [155, 205, 92, 16, 217]
258 [258, 223, 232, 29, 113]
136 [136, 37, 92, 19, 143]
76 [76, 10, 16, 205, 249]
-1 [78, 205, 16, 21, 12]
-1 [283, 218, 264, 135, 206]
53 [53, 238, 276, 25, 93]
-1 [197, 81, 35, 119, 93]
12 [12, 221, 21, 105, 187]
11 [24, 259, 170, 186, 245]
280 [280, 235, 31, 133, 4]
182 [182, 57, 31, 281, 252]
252 [252, 248, 192, 259, 51]
-1 [16, 205, 241, 197, 78]
272 [272, 232, 116, 246, 139]
117 [117, 104, 60, 51, 42]
-1 [140, 250, 78, 60, 119]
242 [242, 74, 157, 221, 107]
260 [260, 29, 174, 163, 147]
94 [94, 225, 192, 215, 248]
132 [132, 218, 137, 9, 57]
223 [223, 58, 91, 160, 259]
-1 [78, 221, 12, 276, 25]
127 [127, 11, 104, 40, 239]
47 [47, 100, 102, 279, 184]
13 [47, 100, 102, 77, 261]
-1 [135, 218,

100%|██████████| 30/30 [00:00<00:00, 429.23it/s]


-1 [264, 45, 259, 186, 29]
48 [48, 244, 116, 106, 119]
251 [251, 96, 109, 51, 174]
32 [32, 11, 161, 82, 88]
161 [161, 40, 137, 11, 81]
81 [81, 60, 241, 40, 239]
62 [30, 62, 64, 129, 79]
38 [38, 52, 264, 101, 45]
-1 [264, 259, 29, 101, 45]
128 [128, 176, 107, 46, 105]
155 [155, 217, 41, 205, 60]
258 [258, 223, 232, 58, 91]
136 [136, 37, 92, 174, 226]
76 [76, 249, 43, 191, 142]
-1 [205, 16, 78, 265, 82]
-1 [283, 135, 45, 218, 277]
53 [53, 54, 198, 254, 145]
-1 [81, 197, 35, 156, 146]
12 [12, 105, 221, 21, 187]
11 [259, 170, 186, 24, 118]
280 [280, 4, 31, 243, 30]
182 [182, 60, 174, 281, 133]
252 [252, 248, 192, 215, 51]
-1 [205, 16, 81, 241, 104]
272 [272, 232, 277, 154, 11]
117 [117, 60, 233, 225, 51]
-1 [31, 82, 205, 78, 265]
242 [242, 157, 176, 202, 221]
260 [260, 29, 174, 271, 163]
94 [94, 225, 192, 261, 100]
132 [132, 57, 218, 137, 103]
223 [223, 91, 58, 57, 252]
-1 [221, 12, 78, 105, 156]
127 [127, 11, 40, 60, 174]
47 [47, 100, 102, 192, 24]
13 [100, 102, 47, 261, 77]
-1 [283, 135,

  0%|          | 0/30 [00:00<?, ?it/s]

Combinación inválida {'shape': (90, 90), 'pixels_per_cell': (17, 17), 'cells_per_block': (8, 8)} debido a error: The input image is too small given the values of pixels_per_cell and cells_per_block. It should have at least: 136 rows and 136 cols.
Búsqueda de parámetros completada. Resultados guardados en 'mejores_resultados.txt'.


#### DAISY

In [ ]:
from skimage.feature import daisy

def daisy_descriptor(image, step=250, radius=50, rings=3, histograms=6, orientations=8):
    # Convertimos la imagen a escala de grises
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculamos los descriptores DAISY sin visualización
    descriptors, dsc_image = daisy(
        gray_image,
        step=step,
        radius=radius,
        rings=rings,
        histograms=histograms,
        orientations=orientations,
        visualize=True
    )
    
    return descriptors, dsc_image


In [ ]:
def daisy_descriptor(image, step=158, radius=50, rings=3, histograms=6, orientations=8):
    # Convertimos la imagen a escala de grises
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Usamos un detector de keypoints (aquí FAST, pero puedes usar otros)
    fast = cv2.FastFeatureDetector_create()
    keypoints = fast.detect(gray_image, None)
    
    # Calculamos los descriptores DAISY en los puntos clave detectados
    descriptors = daisy(
        gray_image,
        step=step,
        radius=radius,
        rings=rings,
        histograms=histograms,
        orientations=orientations,
        visualize=False
    )
    
    # Alineamos los descriptores con los keypoints (si es necesario)
    keypoints_daisy = []
    keypoint_descriptors = []
    
    for kp in keypoints:
        # Para cada keypoint, calculamos su posición (en coordenadas de píxeles)
        y, x = kp.pt
        x, y = int(x), int(y)
        
        # Aquí usamos una verificación simple, que si el keypoint está dentro de la imagen
        if x < descriptors.shape[1] and y < descriptors.shape[0]:
            keypoints_daisy.append(kp)
            keypoint_descriptors.append(descriptors[y, x, :])  # Tomamos el descriptor en esa ubicación
    
    # Convertimos las listas a arrays de numpy para facilitar el uso
    keypoints_daisy = np.array(keypoints_daisy)
    keypoint_descriptors = np.array(keypoint_descriptors)
    
    return keypoints_daisy, keypoint_descriptors

In [ ]:
daisy_query = []
for picture in tqdm(QSD1_w4_filtered[:4]):
    for painting in picture:
        kp, des = daisy_descriptor(painting)
        daisy_query.append({'kp': kp, 'des': des})


daisy_bd = []
for picture in tqdm(QSD1_w4_nonAugmented[:4]):
    for painting in picture:
        kp, des = daisy_descriptor(painting)
        daisy_bd.append({'kp': kp, 'des': des})

In [ ]:
# Mostrar la imagen original y la visualización del descriptor DAISY
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray')
plt.title('Imagen original')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(dsc_image, cmap='gray')
plt.title('Descriptor DAISY')
plt.axis('off')

plt.show()

# 2
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(image1, cmap='gray')
plt.title('Imagen original')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(dsc_image1, cmap='gray')
plt.title('Descriptor DAISY')
plt.axis('off')

plt.show()

In [ ]:
# Convertir descriptores a tipo float32 para el matcher de OpenCV
descriptors = des_daisy.astype(np.float32).reshape(-1, 152)
descriptors1 = des_daisy1.astype(np.float32).reshape(-1, 152)

# Crear el matcher Brute-Force con la métrica de distancia Euclidiana
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

# Encontrar las mejores coincidencias
matches = bf.match(descriptors, descriptors1)

# Ordenar las coincidencias por distancia (las más cercanas son las mejores)
matches = sorted(matches, key=lambda x: x.distance)

# Para dibujar los keypoints necesitamos crear un arreglo de KeyPoint
# Generar keypoints para la primera y segunda imagen a partir de la malla DAISY
keypoints = []
keypoints1 = []

step = 158  # El valor del 'step' usado en DAISY
radius = 50  # El radio del descriptor

# Generar keypoints para la imagen original
for i in range(des_daisy.shape[0]):
    for j in range(des_daisy.shape[1]):
        keypoints.append(cv2.KeyPoint(j * step + radius, i * step + radius, 1))

# Generar keypoints para la segunda imagen
for i in range(des_daisy1.shape[0]):
    for j in range(des_daisy1.shape[1]):
        keypoints1.append(cv2.KeyPoint(j * step + radius, i * step + radius, 1))

# Dibujar los matches entre las imágenes
img_matches = cv2.drawMatches(
    image, keypoints,  # Imagen original y sus keypoints
    image1, keypoints1,  # Imagen de referencia y sus keypoints
    matches[:30], None
)

# Mostrar el resultado con matplotlib
plt.figure(figsize=(15, 10))
plt.imshow(img_matches)
plt.title('Coincidencias entre imagen 1 y imagen 2 usando DAISY')
plt.axis('off')  # Desactivar los ejes para que sea más limpio
plt.show()



In [ ]:
surf_query = []
for picture in tqdm(QSD1_w4_filtered):
    for painting in picture:
        kp, des = surf_descriptor(painting)
        surf_query.append({'kp': kp, 'des': des})


surf_bd = []
for picture in tqdm(QSD1_w4_nonAugmented):
    for painting in picture:
        kp, des = surf_descriptor(painting)
        surf_bd.append({'kp': kp, 'des': des})

#### ORB

In [ ]:
def orb_descriptor(image, params={}):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(**params)
    kp, des = orb.detectAndCompute(img_gray, None)

    return (kp, des)

# Task 2

### a) Find tentative matches based on similarity of local appearance and verify matches 

### b) Implement a system to discard queries not in the data set (unknowns)

# Task 3

### a) Evaluate system based on keypoint descriptors on QSD1-W4

### b) Compare your best query system from previous week on QSD1-W4

# Task 4

### a) Create pkl file